In [ ]:
!pip install datasets pandas pymongo sentence_transformers

In [ ]:
!pip install -U transformers accelerate

In [ ]:
from datasets import load_dataset

In [ ]:
import pandas as pd

In [ ]:
dataset=load_dataset("AIatMongoDB/embedded_movies")

In [ ]:
dataset

In [ ]:
dataset_df=pd.DataFrame(dataset["train"])

In [ ]:
dataset_df.head()

In [ ]:
dataset_df.columns

In [ ]:
dataset_df["plot"][0]

In [ ]:
dataset_df["fullplot"][0]

In [ ]:
dataset_df["num_mflix_comments"][0]

In [ ]:
dataset_df[o"fullplt"].isnull().sum()

In [ ]:
dataset_df.shape

In [ ]:
dataset_df["poster"][0]

In [ ]:
dataset_df["plot_embedding"][0]

In [ ]:
dataset_df.isnull().sum()

In [ ]:
dataset_df=dataset_df.dropna(subset=["fullplot"])

In [ ]:
dataset_df["fullplot"].isnull().sum()

In [ ]:
dataset_df = dataset_df.drop(columns=["plot_embedding"])

In [ ]:
dataset_df.head(2)

In [ ]:
# @title metacritic

from matplotlib import pyplot as plt
dataset_df['metacritic'].plot(kind='hist', bins=20, title='metacritic')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("thenlper/gte-large")

In [ ]:
dataset_df["fullplot"][2]

In [ ]:
text="   savita is  a data scientist who create prodcut of data"

In [ ]:
text="   savita is  a data scientist who create prodcut of data     "


In [ ]:
text

In [ ]:
text.strip()

In [ ]:
def get_embedding(text:str)->list[float]:

  if not text.strip():
    print("attempted to get embedding for empty text.")
    return []

  embedding=embedding_model.encode(text)
  return embedding.tolist()


In [ ]:
dataset_df["embedding"]=dataset_df["fullplot"].apply(get_embedding)

In [ ]:
dataset_df.head(3)

In [ ]:
dataset_df["embedding"][0]

In [ ]:
!python --version

In [ ]:
import pymongo

In [ ]:
!python -m pip install "pymongo[srv]"


In [ ]:
!pip list

In [ ]:
from pymongo.mongo_client import MongoClient

In [ ]:
from google.colab import userdata
uri=userdata.get('MONGO_URI')

In [ ]:
# Create a new client and connect to the server
client = MongoClient(uri)

In [ ]:
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
def get_mongo_client(uri):
  try:
    client = MongoClient(uri)
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
    return client
  except Exception as e:
    print(e)
    return None

In [ ]:
mongo_client=get_mongo_client(uri)

In [ ]:
db=mongo_client["moviedb"]

In [ ]:
collection=db["moviecollection"]

In [ ]:
collection.insert_one({"name":"sun",
                       "designation": "genai engineer",
                       "location":"bangaluru",
                       "mailid":"sun.savi@ineon.ai"})

In [ ]:
collection.insert_one({"name":"dip",
                       "designation": "ops manager",
                       "location":"bangaluru"})

In [ ]:
collection2=db["moviecollectionsecond"]

In [ ]:
collection2.insert_one({"name":"kris",
                       "designation": "tech lead",
                       "location":"bangaluru",
                        "phonenumber":57454745834})

In [ ]:
collection.delete_many({})

In [ ]:
dataset_df.tail(3)

In [ ]:
document=dataset_df.to_dict("records")

In [ ]:
collection.insert_many(document)

print("data ingestion in mongodb is completed")

In [ ]:
{
 "fields": [{
     "numDimensions": 1024,
     "path": "embedding",
     "similarity": "cosine",
     "type": "vector"
   }]
}

In [ ]:
def vector_search(user_query,collection):

  query_embedding=get_embedding(user_query)

  if query_embedding is None:
    return "Invalid query or embeddig is failed"

  pipeline=[

            {
                "$vectorSearch":{

                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 4,  # Return top 4 matches


                }

            },

              {
                 "$project":{

                "_id": 0,  # Exclude the _id field
                "fullplot": 1,  # Include the plot field
                "title": 1,  # Include the title field
                "genres": 1,  # Include the genres field
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
                 }

            }

           ]

  result=collection.aggregate(pipeline)
  return list(result)


In [ ]:
list(vector_search("what is the best horror movie to watch and why?",collection))

In [ ]:
query="what is the best horror movie to watch and why?"

In [ ]:
def get_search_result(query,collection):

  get_knowledge=vector_search(query,collection)

  search_result=""

  for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

  return search_result



In [ ]:
query="what is the best horror movie to watch and why?"

source_information=get_search_result(query,collection)

In [ ]:
source_information

In [ ]:
combined_information = f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."

print(combined_information)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")

In [ ]:
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

In [ ]:
# Moving tensors to GPU
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")

In [ ]:
response = model.generate(**input_ids, max_new_tokens=500)

In [ ]:
print(tokenizer.decode(response[0]))